<a href="https://colab.research.google.com/github/aomidvar/scrapper/blob/main/AlienValut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import bs4 as bs
import json
import time
from google.colab import drive

drive.mount('/content/drive')


list_of_att = []
for page in range(16,30):
    print(page)
    a = []
    req = requests.get("https://cybersecurity.att.com/blogs/labs-research/P"+str(page))
    html_page = req.text

    soup = BeautifulSoup(html_page, "lxml")

    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))


    for l in links:
        if type(l) == str:

            if '/labs-research/' in l and 'http' not in l :
                a.append(l)

    a = list(dict.fromkeys(a))

    for aa in a :
        try:
            rr = requests.get('https://cybersecurity.att.com'+aa)
        except:
            rr = requests.get('https://cybersecurity.att.com' + aa)
        parsed_article = bs.BeautifulSoup(rr.text, 'lxml')
        paragraphs = parsed_article.find_all('p')
        article_text = ""

        for p in paragraphs:
            article_text += p.text
        processed_article = article_text.lower()
        s = aa.split('/')
        tittle = s[len(s)-1]
        tittle = tittle.replace('-',' ')
        dict = {
            'titel' : tittle,
            'Explanation' : processed_article
        }
        print(dict)
        list_of_att.append(dict)

with open("/content/drive/MyDrive/allien.json", "a",encoding='utf-8') as outfile:
    json.dump(list_of_att,outfile,ensure_ascii=False,indent=4)

Mounted at /content/drive
16
{'titel': 'prism attacks fly under the radar', 'Explanation': 'powered byat&t alien labsat&t alien labs has recently discovered a cluster of linux elf executables that have low or zero anti-virus detections in virustotal (see example in figure 1), though our internal threat analysis systems have flagged them as malicious. \xa0upon inspection of the samples, alien labs has identified them as modifications of the open-source prism backdoor used by multiple threat actors in various campaigns.we have conducted further investigation of the samples and discovered that several campaigns using these malicious executables have managed to remain active and under the radar for more than 3.5 years. the oldest samples alien labs can attribute to one of the actors date from the 8th of november, 2017.figure 1. prism sample marked as clean in virustotal, as captured by alien labs.waterdropthe waterdrop variant is easily identifiable as it includes a function named xencrypt

In [ ]:
import json
if __name__ == '__main__':
    with open('/content/drive/MyDrive/allien.json') as json_file:
        data = json.load(json_file)
    for att in data:
        print(att)

In [ ]:
from IOC_EX.iocext import ioc_find
import json

if __name__ == '__main__':
    list_of_att_ioc = []
    with open('allien.json') as json_file:
        data = json.load(json_file)
    for att in data:
        iocs = ioc_find(att['Explanation'])
        list_of_att_ioc.append({
            'titel' : att['titel'],
            'explanation' : att['Explanation'],
            'ioc' : iocs
        })
    print(list_of_att_ioc)

    json_object = json.dumps(list_of_att_ioc, indent=4,ensure_ascii=False)

    with open("./content/drive/MyDrive/alien_ioc.json", "a") as outfile:
        outfile.write(json_object)